# ABS Monthly Consumer Price Index Indicator 6484

Caution - the month;y indicator is problematic, and easily misunderstood
 * The CPI indicator is not the whole basket of goods found in the CPI
 * Some included items are only updated quarterly or annually - these items overly inflate the months in which they appear, 
   and do no contribute anything in the months they do not appear.
 * Some volatile items (especially travel) are included in the series that excluded volatile items.
 * The monthly series is way more noisy than the quarterly CPI series.
 * Annualising less than 12 month changes is problematic, and should be avoided.
 * Strongly advised: Focus on the annual trimmed mean indicator. Just ignore month-to-month changes in all series. 
   Favour the seasonally adjusted headline series over the original headline series. 

## Python set-up

In [1]:
# analytic imports
import pandas as pd
import readabs as ra
from readabs import metacol as mc
import textwrap

# local imports
from mgplot import (
    calc_growth,
    finalise_plot,
    line_plot_finalise,
    series_growth_plot_finalise,
    multi_start,
)
from abs_helper import get_abs_data, ANNUAL_CPI_TARGET_RANGE, MONTHLY_CPI_TARGET
from henderson import hma

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

In [2]:
abs_dict, meta, source, _ = get_abs_data("6484.0")


## Plot

### Plot growth - combined line/bar charts

In [3]:
def cpi_growth():
    """Produce CPI growth charts."""

    twrap = textwrap.TextWrapper(width=70)
    rows = meta[meta[mc.did].str.contains("Index Numbers")]

    long_name = 50
    for index, row in rows.iterrows():
        # get data for the plot
        did, table = row[mc.did], row[mc.table]
        series_type, series_id = row[mc.stype], row[mc.id]
        name = did.split(";")[1].strip()
        series = abs_dict[table][series_id]
        separator = " " if len(name) < long_name else "\n"
        multi_start(
            series,
            function=series_growth_plot_finalise,
            starts=[0, -19],
            title=twrap.fill((f"Price Growth:{separator}{name}")),
            rfooter=source,
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            axhline=MONTHLY_CPI_TARGET,
             legend=True,
            lfooter=f"Australia: {series_type.capitalize()} series. Monthly data.",
            y0=True,
            zero_y=True,
            show=SHOW,
        )

        # --- six months annualised growth
        growth = pd.DataFrame()
        for months, label in zip((6, 12), ("Six monnths annualised", "Annual growth")):
            # calculate the growth - using compount annual growth rates
            rate = ((series / series.shift(periods=months)) ** (1 / (months/12)) - 1) * 100.
            growth[label] = rate
        line_plot_finalise(
            growth,
            plot_from=-19,
            title=twrap.fill((f"Six months annualised growth:{separator}{name}")),
            ylabel="Per cent",
            rfooter=source,
            axhspan=ANNUAL_CPI_TARGET_RANGE,
            legend=True,
            lfooter=f"Australia: {series_type.capitalize()} series. Monthly data.",
            y0=True,
            zero_y=True,
            annotate=[True, False],
            width=[2.5, 1],
            show=SHOW,
        )


cpi_growth()

### Selected sub items

In [4]:
def trimmed_mean():
    """Produces a chart for trimmed mean CPI."""

    name = "Annual Trimmed Mean"
    rows = meta[meta[mc.did].str.contains(name)]
    series_id = rows.iloc[0][mc.id]
    stype = rows.iloc[0][mc.stype]
    table = rows.iloc[0][mc.table]
    data = abs_dict[table][series_id]
    data.name = name
    line_plot_finalise(
        data,
        title=f"{name} CPI Growth",
        ylabel="Per cent",
        axhspan=ANNUAL_CPI_TARGET_RANGE,
        rfooter=source,
        legend=True,
        lfooter=f"Australia: {stype} series. Monthly data. ",
        width=2,
        show=SHOW,
    )

    n = 13
    h = hma(data.dropna(), n)
    line_plot_finalise(
        pd.DataFrame(
            {
                name: data,
                f"{n}-term Henderson Moving Average": h,
            }
        ),
        title=f"{name} CPI Growth (smoothed)",
        ylabel="Per cent",
        axhspan=ANNUAL_CPI_TARGET_RANGE,
        rfooter=source,
        legend={"loc": "best", "ncol": 1, "fontsize": "x-small"},
        lfooter=f"Australia: {stype} series.  Monthly data. ",
        width=(0.75, 2.5),
        annotate=[True, False],
        show=SHOW,
    )


trimmed_mean()

In [5]:
def food_fuel() -> None:
    """Produce food and fuel sub-item charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Food and non-alcoholic beverages": {
            "Food and non-alcoholic beverages": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
        "Automotive fuel": {
            "Automotive fuel": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        found = ra.search_abs_meta(
            meta, selector, verbose=False, validate_unique=True
        ).iloc[0]
        table, series_id = found[mc.table], found[mc.id]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Food & Fuel"
    tag = "1"
    series_type = "Original series"
    line_plot_finalise(
        data,
        title=title,
        ylabel="Per cent",
        width=(1, 2),
        tag=tag,
        legend={"loc": "best", "ncol": 1, "fontsize": "x-small"},
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}. Monthly data. ",
        y0=True,
        axhspan=ANNUAL_CPI_TARGET_RANGE,
        show=SHOW,
    )


food_fuel()

In [6]:
def housing() -> None:
    """Produce housing related charts."""

    annual = "Corresponding Month of Previous Year"
    selectors = {
        "Rents": {
            "Rents": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
        "New dwelling purchase by owner-occupiers": {
            "New dwelling purchase by owner-occupiers": mc.did,
            "Original": mc.stype,
            annual: mc.did,
        },
    }

    data = pd.DataFrame()
    for name, selector in selectors.items():
        found = ra.search_abs_meta(
            meta, selector, verbose=False, validate_unique=True
        ).iloc[0]
        table, series_id = found[mc.table], found[mc.id]
        data[name] = abs_dict[table][series_id]

    title = "Monthly CPI Indicator - Annual Growth - Residential"
    tag = "2"
    series_type = "Original series"
    line_plot_finalise(
        data,
        title=title,
        ylabel="Per cent",
        width=(1, 2),
        tag=tag,
        rfooter=f"{source}",
        lfooter=f"Australia: {series_type}. Monthly data. ",
        y0=True,
        legend={"loc": "best", "ncol": 1, "fontsize": "x-small"},
        axhspan=ANNUAL_CPI_TARGET_RANGE,
        show=SHOW,
    )


housing()

## Finished

In [7]:
# watermark
%load_ext watermark
%watermark -u -t -d --iversions --watermark --machine --python --conda

Last updated: 2025-09-04 08:19:51

Python implementation: CPython
Python version       : 3.13.6
IPython version      : 9.4.0

conda environment: n/a

Compiler    : Clang 20.1.4 
OS          : Darwin
Release     : 24.6.0
Machine     : arm64
Processor   : arm
CPU cores   : 14
Architecture: 64bit

mgplot : 0.2.12
readabs: 0.1.4
pandas : 2.3.1

Watermark: 2.5.0



In [8]:
print("Done")

Done
